In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
from labrotation.two_photon_session import TwoPhotonSession
import labrotation.file_handling as fh
from matplotlib import pyplot as plt
import matplotlib
import os
import h5py
import pims_nd2
import numpy as np
import math
import h5py
import pyabf
from datetime import datetime
import warnings
import pandas as pd

### Set output folder

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
output_folder = env_dict["DOWNLOADS_FOLDER"]

### Set parameters

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 20}

matplotlib.rc('font', **font)

# Open file

In [ ]:
nd2_fpath = fh.open_file("Open nikon nd2 file!")
nik_tstamps_fpath = fh.open_file("Open utf-8 nikon _nik.txt time stamp file!")  # need it for precise time stamps; Nikon gets the imaging frequency wrong sometimes...
lfp_fpath = fh.open_file("Open LFP .abf file!")

In [ ]:
df_tstamps = pd.read_csv(nik_tstamps_fpath, encoding="utf-8", delimiter="\t")

In [ ]:
nik_data = pims_nd2.ND2_Reader(nd2_fpath)

# Limit to specific pixels
The assumption is that even though the voltage indicator has negative signal (i.e. activity = lower fluorescence level), the baseline is higher where there is expression.  

In [ ]:
PIXEL_PERCENT = 0.05  # 0.1 corresponds to 10%, i.e. take 10% of total pixels for trace extraction

res = nik_data[0].shape
n_pixels = res[0]*res[1]

n_selected_pixels = math.ceil(n_pixels*PIXEL_PERCENT)

pixel_percent_100 = PIXEL_PERCENT*100  # in %, i.e. max is 100

In [ ]:
n_frames_template = 10

In [ ]:
baseline_img = np.array(nik_data[0:n_frames_template]).mean(axis=0)

In [ ]:
fig = plt.figure(figsize=(18,18))
plt.imshow(baseline_img)
plt.show()

## Set geometrical limits to soma

In [ ]:
X_MIN = 100#200  # inclusive
X_MAX = 400#325  # exclusive
Y_MIN = 0    # inclusive
Y_MAX = 100#32   # exclusive

In [ ]:
if "X_MIN" not in locals():
    X_MIN = 0
if "X_MAX" not in locals():
    X_MAX = baseline_img.shape[1]
if "Y_MIN" not in locals():
    Y_MIN = 0
if "Y_MAX" not in locals():
    Y_MAX = baseline_img.shape[0] 

In [ ]:
baseline_img_cropped = baseline_img[Y_MIN:Y_MAX,X_MIN:X_MAX]

## Check FOV to be used

In [ ]:
fig = plt.figure(figsize=(18,18))
plt.imshow(baseline_img_cropped)
plt.show()

## Get pixels coordinates

In [ ]:
# flatten the baseline image, sort indices by value. Need to flip it from ascending to 
# descending order: then first n elements will be the largest (i.e. brightest pixels)
selected_pixels = np.flip(np.argsort(baseline_img_cropped.flatten()))[:n_selected_pixels]
# unravel into the 2d indexing
row_indices, col_indices = np.unravel_index(selected_pixels, baseline_img_cropped.shape)

### Convert back indices to non-cropped shape

In [ ]:
row_indices += Y_MIN
col_indices += X_MIN

## Check resulting pixels
Yellow/green marks the selected pixels, contrasted with dark blue background

In [ ]:
binary_img = np.zeros(baseline_img.shape)
binary_img[row_indices,col_indices] = 1.0

In [ ]:
fig = plt.figure(figsize=(18,18))
plt.imshow(baseline_img)
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,18))
plt.imshow(binary_img)
plt.show()

In [ ]:
fov_shape = binary_img.shape

# Extract mean trace of selected pixels

In [ ]:
# [:, row_indices, col_indices] flattens the array into shape (n_frames, n_selected_pixels)
y_nikon = np.array(nik_data)[:,row_indices, col_indices].mean(axis=1)

In [ ]:
tstamp_start_nik = nik_data.metadata["time_start"]

In [ ]:
# extract lfp 
lfp = pyabf.ABF(lfp_fpath)

In [ ]:
tstamp_start_lfp = lfp.abfDateTime

In [ ]:
df_tstamps["Events"].unique()

In [ ]:
df_tstamps_filtered = df_tstamps[df_tstamps["Events"].isna()]

In [ ]:
t_nikon = np.array(df_tstamps_filtered["SW Time [s]"])
t_nikon = t_nikon - t_nikon[0]

In [ ]:
# assume nikon was started after LFP
delay_nik = (tstamp_start_nik - tstamp_start_lfp).total_seconds() 
if delay_nik < 0:
    warnings.warn("Nikon seems to have started before LFP!")

In [ ]:
LFP_SCALING_FACTOR = 1.0038  # the axoscope time stamps are off by a constant factor...

In [ ]:
lfp.setSweep(0, 0)
t_lfp = lfp.sweepX * LFP_SCALING_FACTOR
y_lfp = lfp.sweepY
lfp.setSweep(0, 1)
t_loco = lfp.sweepX * LFP_SCALING_FACTOR
y_loco = lfp.sweepY

In [ ]:
delay_lfp_manual = 0.48#0.4  # adjust this: increase to delay LFP, decrease to bring it earlier 

In [ ]:
current_time = datetime.now()

In [ ]:
fname_out_fig = os.path.splitext(os.path.split(nd2_fpath)[-1])[0] + "_fig_" + current_time.strftime("%Y%m%d_%H%M%S") + ".pdf"
fpath_out_fig = os.path.join(output_folder, fname_out_fig)
print(f"Saving figure to:\n\t{fpath_out_fig}")

In [ ]:
fig, axs = plt.subplots(3, 1, sharex=True, figsize = (18, 18))
plt.suptitle(os.path.split(nd2_fpath)[-1])
axs[0].plot(t_nikon, y_nikon, color="green", label="fluo.", linewidth=0.5)
axs[1].plot(t_lfp - delay_nik + delay_lfp_manual, y_lfp, color="blue", label="lfp", linewidth=0.5)
axs[2].plot(t_loco - delay_nik + delay_lfp_manual, y_loco, color="grey", label="loco.", linewidth=0.5)
#plt.xlim((20, 22.5))
plt.savefig(fpath_out_fig)
plt.show()

In [ ]:
current_time = datetime.now()

In [ ]:
fname_out = os.path.splitext(os.path.split(nd2_fpath)[-1])[0] + "_" + current_time.strftime("%Y%m%d_%H%M%S") + ".h5"
fpath_out = os.path.join(output_folder, fname_out)
print(f"Saving to:\n\t{fpath_out}")

In [ ]:
# save all time series and matching parameters (shift values, scaling factor...)
with h5py.File(fpath_out, "w") as hf:
    hf.create_dataset("y_nikon", data=y_nikon)
    hf.create_dataset("t_nikon", data=t_nikon)
    hf.create_dataset("t_lfp", data=t_lfp - delay_nik + delay_lfp_manual)
    hf.create_dataset("y_lfp", data=y_lfp)
    hf.create_dataset("t_loco", data=t_loco - delay_nik + delay_lfp_manual)
    hf.create_dataset("y_loco", data=y_loco)
    hf.create_dataset("used_pixels_for_y_nikon", data=selected_pixels)
    hf.create_dataset("used_pixels_col", data=col_indices)
    hf.create_dataset("used_pixels_row", data=row_indices)
    hf.create_dataset("binary_template_image", data=binary_img)
    hf.create_dataset("template_image", data=baseline_img)
    hf.attrs["fov_shape"] = fov_shape
    hf.attrs["nikon_fname"] = os.path.split(nd2_fpath)[-1]
    hf.attrs["nikon_time_stamps_fname"] = os.path.split(nik_tstamps_fpath)[-1]
    hf.attrs["lfp_fname"] = os.path.split(lfp_fpath)[-1]
    hf.attrs["t_lfp_scaling_factor"] = LFP_SCALING_FACTOR
    hf.attrs["scaling_factor_comment"] = "loco and lfp from axoscope need scaling for time steps"
    hf.attrs["delay_nikon_to_lfp"] = delay_nik
    hf.attrs["delay_lfp_manual"] = delay_lfp_manual
    hf.attrs["n_frames_for_template"] = n_frames_template
    hf.attrs["window_XMIN"] = X_MIN
    hf.attrs["window_XMAX"] = X_MAX
    hf.attrs["window_YMIN"] = Y_MIN
    hf.attrs["window_YMAX"] = Y_MAX
    
    
    

# Check if reading file works

In [ ]:
with h5py.File(fpath_out, "r") as hf:
    fov_shape = hf.attrs["fov_shape"]
    tn = hf["t_nikon"][()]
    yn = hf["y_nikon"][()]
    tlfp = hf["t_lfp"][()]
    ylfp = hf["y_lfp"][()]
    tloc = hf["t_loco"][()]
    yloc = hf["y_loco"][()]
    template = hf["template_image"][()]
    pixels_row = hf["used_pixels_row"][()]
    pixels_col = hf["used_pixels_col"][()]
    binary_template = hf["binary_template_image"][()]

In [ ]:
plt.imshow(binary_template)

In [ ]:
check_binary_template = np.zeros(fov_shape)
check_binary_template[pixels_row, pixels_col] = 1
plt.imshow(check_binary_template)

In [ ]:
plt.imshow(template)

In [ ]:
fig, axs = plt.subplots(3, 1, sharex=True, figsize = (18, 18))
plt.suptitle(os.path.split(nd2_fpath)[-1])
axs[0].plot(tn, yn, color="green", label="fluo.", linewidth=0.5)
axs[1].plot(tlfp, ylfp, color="blue", label="lfp", linewidth=0.5)
axs[2].plot(tloc, yloc, color="grey", label="loco.", linewidth=0.5)
#plt.xlim((0, 50))
plt.show()

# Compare with whole FOV mean trace

In [ ]:
mean_trace_whole_fov = np.array(nik_data).mean(axis=(1,2))

In [ ]:
mean_trace_whole_fov.shape

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1,figsize=(18,12), sharex=True, sharey=False)

axs[0].plot(mean_trace)
axs[0].title.set_text(f'Brightest {pixel_percent_100}% pixels mean')

axs[1].plot(mean_trace_whole_fov)
axs[1].title.set_text('Whole FOV mean')

axs[0].set_xlim((9500, 30000))
axs[1].set_xlim((9500, 30000))

axs[0].set_ylim((40,95))
axs[1].set_ylim((35,60))

axs[0].set_ylabel('fluorescence (a.u.)')
axs[1].set_ylabel('fluorescence (a.u.)')

axs[1].set_xlabel("frame")

plt.show()

# Create summary plot
For reporting results per recording

In [ ]:
save_summary_fig = False


fig, axs = plt.subplots(nrows=4, ncols=1,figsize=(18,24), sharex=False, sharey=False, gridspec_kw={'height_ratios': [3, 3, 1, 1]})

axs[0].plot(mean_trace)
axs[0].title.set_text(f'Brightest {int(pixel_percent_100)}% pixels mean')

axs[1].plot(mean_trace_whole_fov)
axs[1].title.set_text('Whole FOV mean')

axs[0].set_xlim((2000,12000))#((9500, 30000))
axs[1].set_xlim((2000,12000))#((9500, 30000))
axs[0].set_ylim((40,95))
axs[1].set_ylim((35,60))

axs[0].set_ylabel('fluorescence (a.u.)')
axs[1].set_ylabel('fluorescence (a.u.)')

axs[1].set_xlabel("frame")


axs[2].imshow(baseline_img)
axs[2].title.set_text("FOV image")


axs[3].title.set_text(f"Brightest {int(pixel_percent_100)}% pixels")
axs[3].imshow(binary_img)
plt.tight_layout()
if save_summary_fig:
    output_fpath = os.path.join(env_dict["DOWNLOADS_FOLDER"], f"voltage_comparison_{fh.get_datetime_for_fname()}.jpg")
    plt.savefig(output_fpath)
    print(f"Saved to {output_fpath}")
plt.show()

In [ ]:
'''
# Add data to already existing file
with h5py.File(, "a") as hf:
    hf["mean_fluo_nucleus"] = mean_trace
    hf["fov"] = baseline_img
    hf["selected_pixels"] = binary_img
'''